In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from skimage.data import imread
import matplotlib.pyplot as plt
import torch.nn as nn 
import torch.nn.functional as F
import torch
from inceptionresnetv2.pytorch_load import InceptionResnetV2
from pathlib import Path

USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial
# the default is gpu but if gpu is not available,then use cpu
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

ship_dir = '/datasets/ee285f-public/airbus_ship_detection/'
train_image_dir = os.path.join(ship_dir, 'train_v2')
test_image_dir = os.path.join(ship_dir, 'test_v2')
label = pd.read_csv(os.path.join(ship_dir,'/datasets/ee285f-public/airbus_ship_detection/train_ship_segmentations_v2.csv'))
#dataset_train = '/datasets/ee285f-public/airbus_ship_detection/'
#csv_train = 	'/datasets/ee285f-public/airbus_ship_detection/train_ship_segmentations_v2.csv''

    # read_csv_file
print("frame： ",label.shape[0])

    # delete annotations without ship
delete = label.dropna(axis=0)
    #num_of_ships = df.shape[0]
have_ships = label.shape[0]
print("instance      ： ",have_ships)

    # store images with ship in another set
images_with_ship = set()
for num in range(have_ships):
    if label.iloc[num,0] not in images_with_ship:
        images_with_ship.add(label.iloc[num,0])
print("have ship： ",len(images_with_ship))

    # Delete images without ship
#count = 0
#lis = os.listdir(ship_dir)
#for j in lis:
#    if j not in images_with_ship:
#        os.remove(os.path.join(ship_dir, j))
#        count += 1
#print('%d images is deleted.'%(count))

In [ ]:
np_la = label.values
np_train = np_la[:,0]
total = len(np_train)
#np_label = np_la[:,2]
def normalize_MNIST_images(x):
    return 2 * x.astype(np.float64) / 255. - 1

In [ ]:
net = InceptionResnetV2().cuda()
criterion = nn.CrossEntropyLoss()

In [ ]:
layer1 = nn.Sequential(
        nn.Conv2d(3, 3, 13)
        nn.MaxPool2d(3, stride=2)
        nn.Conv2d(3, 3, 2),
        nn.Conv2d(3, 3, 3),
        nn.Conv2d(3, 3, 5),
        nn.Conv2d(3, 3, 13),
        nn.Conv2d(3, 3, 13),
        nn.Conv2d(3, 3, 13),
        nn.Conv2d(3, 3, 13),
        nn.Conv2d(3, 3, 13),
        nn.Conv2d(3, 3, 13),
    ),

In [ ]:
model = nn.Sequential(
        layer1,
        net.forward(),
    )

In [ ]:
model_path = Path('InceptionResnetV2.1.tnt')
gamma = 1e-5
optimizer = torch.optim.Adam(net.parameters(), lr=gamma )
if model_path.exists():
    net = torch.load(str(model_path))
    print('load succerssful')

model = model().cuda()
#optimizer = torch.optim.SGD(net.parameters(), lr=gamma , momentum=0.9)
batch_size = 10
epoch = 3
size = 299, 299
load_flag = 0
NB = (total+batch_size-1)//batch_size
print(NB)
false = 0

In [ ]:
false_0 = 0
false_1 = 0
for e in range(1):
    loss_epoch = 0
    i=0
    '''
    with open('accurracy_0.log', 'r') as f:
        if load_flag ==0:
            false_0 = int(f.read())
    with open('accurracy_1.log', 'r') as f:
        if load_flag ==0:
            false_1 = int(f.read())
    with open('test_his.log', 'r') as f:
        if load_flag ==0:
            i = int(f.read())
            load_flag = 1

    '''
    while i < NB:

        x = np.zeros([batch_size,768,768,3])
        y = np.zeros([batch_size])
        #idx = np.arange(i*batch_size:(i+1)*batch_size)
        for j in range(batch_size):

            try:
                img = Image.open(os.path.join(train_image_dir,np_train[(i-1)*batch_size+j]))
                #img = Image.open(os.path.join(train_image_dir,np_train[j]))
                x[j] = np.array(img)
                y[j] = np_label[(i-1)*batch_size+j]
                #y[j] = np_label[j]
            except:
                print(np_train[j])
        #np.savez('save\input_'+str(i)+'.npz',inpu = x)
        x =  np.moveaxis(x, [0,1,2,3], [0,2,3,1])
        x =  normalize_MNIST_images(x)

        x =  torch.from_numpy(x).float()
        y =  torch.from_numpy(y).float()
        #print(im)
        inpu = x.to(device=device, dtype=dtype)  

        ltemp = y.to(device=device, dtype=torch.long)
        optimizer.zero_grad()
        output = model(inpu)
        #print(output)
        scores = nn.functional.softmax(output,dim = 1)
        for n in range(batch_size):
            if scores[n,0]>scores[n,1] and ltemp[n]==1:
                false_1 +=1
            elif scores[n,0]<scores[n,1] and ltemp[n]==0:
                false_0 +=1
                
        #output = torch.exp(output)
        #output = output/torch.sum(output,1) 
        #print(output,scores,ltemp)
        loss = criterion(scores,ltemp)
        #loss = sum((scores[:,1]-ltemp)**2)
        #print(i,false)
        #print(scores,ltemp)
        loss.backward()
        # Parameter update 
        optimizer.step()
        if i%100 == 0:
            #print(i,loss)
        if i%100 ==0:
            
            with open('accurracy_0.log', 'w') as f:
                f.write(str(false_0))
            with open('accurracy_1.log', 'w') as f:
                f.write(str(false_1))
                
            with open('test_his.log', 'w') as f:
                f.write(str(i))
            print('save successful')
            
        i +=1
    print(loss_epoch/NB)
    net.state_dict()